# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'output_data/cities.csv'

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [30]:
%%capture --no-display


# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    size='Humidity',  # Size of points based on humidity
    color='City',  # Optional: color points based on humidity
    alpha=0.6,        # Transparency of points
    tiles='OSM',      # Use OpenStreetMap tiles for the background
    geo=True,          # Treat coordinates as geographic data
    title='Humidity of each city ',  # Title of the map
     frame_width=1000,                        # Width of the plot
    frame_height=600  
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Filter for ideal weather conditions
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] < 40) &
    (city_data_df['Max Temp'] > 35)
    ]

# Drop any rows with null values
ideal_weather_df_cleaned = ideal_weather_df.dropna()
ideal_weather_df_cleaned

#Display a sample of the cleaned data
sample_data = ideal_weather_df_cleaned.sample(n=10) 

sample_data


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
238,238,hinton,53.4001,-117.5857,39.29,81,13,2.28,CA,1727510178
23,23,aasiaat,68.7098,-52.8699,36.84,97,100,6.82,GL,1727509820
272,272,qaqortoq,60.7167,-46.0333,37.60,89,100,1.68,GL,1727510233
91,91,college,64.8569,-147.8028,38.17,92,75,0.00,US,1727509936
562,562,tromso,69.6496,18.9570,39.22,87,40,4.61,NO,1727510713
54,54,ilulissat,69.2167,-51.1000,37.42,100,100,3.44,GL,1727509873
113,113,anadyr,64.7500,177.4833,36.10,80,100,5.99,RU,1727509972
369,369,berdigestyakh,62.0989,126.6992,36.63,84,85,7.45,RU,1727510392
92,92,igarka,67.4667,86.5833,39.42,97,100,13.96,RU,1727509937
68,68,iqaluit,63.7506,-68.5145,38.93,93,75,2.30,CA,1727509897


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng','Max Temp', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
sample_hotel_data = hotel_df.sample(n=10)
sample_hotel_data

,City,Country,Lat,Lng,Max Temp,Humidity,Hotel Name
369,berdigestyakh,RU,62.0989,126.6992,36.63,84,
92,igarka,RU,67.4667,86.5833,39.42,97,
181,susuman,RU,62.7833,148.1667,37.72,71,
117,badger,US,64.8000,-147.5333,37.27,93,
68,iqaluit,CA,63.7506,-68.5145,38.93,93,
562,tromso,NO,69.6496,18.9570,39.22,87,
272,qaqortoq,GL,60.7167,-46.0333,37.60,89,
54,ilulissat,GL,69.2167,-51.1000,37.42,100,
40,udachny,RU,66.4167,112.4000,36.07,62,
91,college,US,64.8569,-147.8028,38.17,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000 
params = { 'categories': 'accommodation.hotel', 
           'limit': 1,                           
           'apiKey': geoapify_key
        
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'# circle
    params["bias"] = f'proximity:{longitude},{latitude}'  

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json() 

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ust-nera - nearest hotel: гостиница "Солнечная"
port-aux-francais - nearest hotel: Keravel
aasiaat - nearest hotel: SØMA
yellowknife - nearest hotel: No hotel found
udachny - nearest hotel: Вилюй
grytviken - nearest hotel: No hotel found
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
whitehorse - nearest hotel: Town & Mountain Hotel
iqaluit - nearest hotel: Frobisher Inn
lobujya - nearest hotel: Hôtel Mother Earth
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
igarka - nearest hotel: Заполярье
anadyr - nearest hotel: Гостевой дом
badger - nearest hotel: Clarion Hotel & Suites
sisimiut - nearest hotel: Hotel SØMA
susuman - nearest hotel: No hotel found
smithers - nearest hotel: Sunshine Inn Hotel
hinton - nearest hotel: Timberland Hotel
qaqortoq - nearest hotel: Hotel Qaqortoq
berdigestyakh - nearest hotel: No hotel found
khandyga - nearest hotel: Апельсин
selfoss - nearest hotel: Miðgarður
inari - nearest hotel: Villa Lanca
tromso - near

,City,Country,Lat,Lng,Max Temp,Humidity,Hotel Name
3,ust-nera,RU,64.5667,143.2000,35.67,75,"гостиница ""Солнечная"""
9,port-aux-francais,TF,-49.3500,70.2167,35.98,64,Keravel
23,aasiaat,GL,68.7098,-52.8699,36.84,97,SØMA
39,yellowknife,CA,62.4560,-114.3525,39.42,96,No hotel found
40,udachny,RU,66.4167,112.4000,36.07,62,Вилюй
42,grytviken,GS,-54.2811,-36.5092,35.20,91,No hotel found
54,ilulissat,GL,69.2167,-51.1000,37.42,100,Best Western Plus Hotel Ilulissat
63,whitehorse,CA,60.7161,-135.0538,38.80,74,Town & Mountain Hotel
68,iqaluit,CA,63.7506,-68.5145,38.93,93,Frobisher Inn
78,lobujya,NP,27.9500,86.8167,35.24,99,Hôtel Mother Earth


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    size='Humidity',  # Size of points based on humidity
    color='City',  # Optional: color points based on humidity
    alpha=0.6,        # Transparency of points
    tiles='OSM',      # Use OpenStreetMap tiles for the background
    geo=True,          # Treat coordinates as geographic data
    title='Hotel Locations by City',  # Title of the map
    xlabel='Longitude',               
    ylabel='Latitude',                 
    frame_width=1000,                       
    frame_height=600,
    hover_cols=['Country', 'Lat', 'Lng','Max Temp', 'Humidity', 'Hotel Name']
)

# Display the map
map_plot_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp,Hotel Name)